# REDCap Datatype and API Export Notes

### Non-Standard Field types
- instrument_name_complete/incomplete

### Standard Field Types
- text
- radio
- dropdown
- file
- descriptive
- notes
- yesno
- checkbox


In addition to standard types, each form (instrument) has a column to indicate if form is complete/incomplete

instrument_name_complete/incomplete is exported via standard API call. 




## API Export Records (default settings)



- text
- radio
- dropdown
- file
- <s> descriptive
- notes
- yesno
- checkbox
- instrument_name_complete/incomplete


"descriptive" field type is NOT exported

# Appendix

## Pycap Methods




### project.export_fields()
    - does NOT export "file" field types
    
### project.export_metadata()
    - does NOT export "instrument_name_complete/incomplete"
    






# Development

# Sratch Sheet